In [1]:
import pandas as pd
from datetime import date, datetime

In [2]:
import sys
sys.setrecursionlimit(10000)

In [3]:
import warnings
warnings.simplefilter("ignore")

In [4]:
df_train = pd.read_csv("purchase_data.csv")

In [5]:
pd.set_option('display.max_columns', None)

In [6]:
df_train.head()

,bill_ref_id,store_ref_id,customer_ref_id,doctor_ref_id,payment_method,created_at_bill,num_drugs_bill,total_quantity_bill,mrp_bill,total_spend_bill,return_value_bill,returned_quantity_bill,quantity_ethical,quantity_generic,quantity_surgical,quantity_ayurvedic,quantity_general,quantity_otc,quantity_chronic,quantity_acute,quantity_h1
0,7562292,468,6251676,6628790,cash,2019-04-01 09:09:51.000000,3,12,394.60,146.92,0.0,0,0,11,1,0,0,0,10,2,0
1,7562335,468,6702206,7298144,cash,2019-04-01 09:30:17.000000,1,1,50.50,26.04,0.0,0,0,1,0,0,0,0,0,1,0
2,7562354,468,6689660,6478435,cash,2019-04-01 09:33:56.000000,3,7,708.50,330.19,0.0,0,0,7,0,0,0,0,0,7,0
3,7562366,468,8539983,6755214,cash,2019-04-01 09:37:13.000000,1,1,52.32,26.00,0.0,0,0,1,0,0,0,0,0,1,0
4,7562370,468,8539984,7298144,cash,2019-04-01 09:38:22.000000,1,1,18.00,14.40,0.0,0,0,1,0,0,0,0,0,1,0


In [7]:
df_train.columns

Index(['bill_ref_id', 'store_ref_id', 'customer_ref_id', 'doctor_ref_id',
       'payment_method', 'created_at_bill', 'num_drugs_bill',
       'total_quantity_bill', 'mrp_bill', 'total_spend_bill',
       'return_value_bill', 'returned_quantity_bill', 'quantity_ethical',
       'quantity_generic', 'quantity_surgical', 'quantity_ayurvedic',
       'quantity_general', 'quantity_otc', 'quantity_chronic',
       'quantity_acute', 'quantity_h1'],
      dtype='object')

In [8]:
df_train.shape

(1184025, 21)

## Dim_Date

In [9]:
def make_dim_date(df):
    
    df["created_at_bill"] = pd.to_datetime(df["created_at_bill"])
    df["Date"] = df["created_at_bill"].map(lambda x: x.strftime('%Y-%m-%d'))
    df["Date"] = pd.to_datetime(df["Date"])
    df["DateID"] = df["Date"].map(lambda x: x.strftime('%Y%m%d'))
    
    # Criando as features da tabela dim_date
    dim_date = df[["DateID", "Date"]]
    dim_date["Full Date Description"] = df["Date"].map(lambda x: x.ctime())
    dim_date["Day Of Week"] = df["Date"].map(lambda x: x.strftime("%A"))
    dim_date["Month"] = df["Date"].map(lambda x: x.strftime("%B"))
    dim_date["Year"] = df["Date"].map(lambda x: x.strftime("%Y"))
    
    dim_date = dim_date.drop_duplicates(subset=["DateID"], keep='first')
    
    df.drop(["created_at_bill"], axis=1, inplace=True)
    
    return dim_date

In [10]:
dim_date = make_dim_date(df_train)

In [11]:
dim_date.shape

(183, 6)

In [12]:
dim_date.head()

,DateID,Date,Full Date Description,Day Of Week,Month,Year
0,20190401,2019-04-01,Mon Apr 1 00:00:00 2019,Monday,April,2019
183,20190402,2019-04-02,Tue Apr 2 00:00:00 2019,Tuesday,April,2019
391,20190403,2019-04-03,Wed Apr 3 00:00:00 2019,Wednesday,April,2019
587,20190404,2019-04-04,Thu Apr 4 00:00:00 2019,Thursday,April,2019
779,20190405,2019-04-05,Fri Apr 5 00:00:00 2019,Friday,April,2019


## Dim_Store

In [13]:
def make_dim_store(df):
    
    df["Store Name"] = df["store_ref_id"].map(lambda x: "Store " + str(x))
    
    dim_store = df[["store_ref_id","Store Name"]]
    dim_store["Store_ref_key"] = df["store_ref_id"]
    dim_store["Address"] = df["store_ref_id"].map(lambda x: "Rua " + str(x))
    dim_store["City"] = df["store_ref_id"].map(lambda x: "Cidade " + str(x))
    dim_store["Region"] = df["store_ref_id"].map(lambda x: "Region " + str(x))
    dim_store["Zip Code"] = df["store_ref_id"].map(lambda x: "0000-" + str(x))
    dim_store["Phone Number"] = df["store_ref_id"].map(lambda x: "+351 000000" + str(x))
    dim_store["Row Effective Date"] = df["Date"]
    dim_store["Row Expiration Date"] = "2999-12-31"
    dim_store["Current Row Indicator"] = 1

    
    dim_store = dim_store.drop_duplicates(subset=["store_ref_id"], keep='first')
    
    df.drop(["Store Name"], axis=1, inplace=True)
    
    return dim_store

In [14]:
dim_store = make_dim_store(df_train)

In [15]:
dim_store.shape

(43, 11)

In [16]:
dim_store.head()

,store_ref_id,Store Name,Store_ref_key,Address,City,Region,Zip Code,Phone Number,Row Effective Date,Row Expiration Date,Current Row Indicator
0,468,Store 468,468,Rua 468,Cidade 468,Region 468,0000-468,+351 000000468,2019-04-01,2999-12-31,1
6430,473,Store 473,473,Rua 473,Cidade 473,Region 473,0000-473,+351 000000473,2019-04-01,2999-12-31,1
17105,471,Store 471,471,Rua 471,Cidade 471,Region 471,0000-471,+351 000000471,2019-04-01,2999-12-31,1
29107,467,Store 467,467,Rua 467,Cidade 467,Region 467,0000-467,+351 000000467,2019-04-01,2999-12-31,1
39176,459,Store 459,459,Rua 459,Cidade 459,Region 459,0000-459,+351 000000459,2019-04-01,2999-12-31,1


## Dim_Customer

In [17]:
def make_dim_customer(df):
    
    df["Customer Name"] = df["customer_ref_id"].map(lambda x: "Customer " + str(x))
                                                  
    dim_customer = df[["customer_ref_id", "Customer Name"]]
    dim_customer["customer_ref_key"] = df["customer_ref_id"]
    dim_customer["NIF"] = df["customer_ref_id"].map(lambda x: "00" + str(x))
    dim_customer["Address"] = df["customer_ref_id"].map(lambda x: "Rua " + str(x))
    dim_customer["City"] = df["customer_ref_id"].map(lambda x: "Cidade " + str(x))
    dim_customer["Region"] = df["customer_ref_id"].map(lambda x:  "Region " + str(x))
    dim_customer["Zip Code"] = df["customer_ref_id"]
    dim_customer["Phone Number"] = df["customer_ref_id"].map(lambda x: "+351 00" + str(x))
    dim_customer["Address E-mail"] = df["customer_ref_id"].map(lambda x: "email" + str(x) + "@gmail.com")
    dim_customer["Row Effective Date"] = df["Date"]
    dim_customer["Row Expiration Date"] = "2999-12-31"
    dim_customer["Current Row Indicator"] = 1
                                                  
    
    dim_customer = dim_customer.drop_duplicates(subset=["customer_ref_id"], keep='first')
    
    df.drop(["Customer Name"], axis=1, inplace=True)
    
    return dim_customer

In [18]:
dim_customer = make_dim_customer(df_train)

In [19]:
dim_customer.shape

(278136, 13)

In [20]:
dim_customer.head()

,customer_ref_id,Customer Name,customer_ref_key,NIF,Address,City,Region,Zip Code,Phone Number,Address E-mail,Row Effective Date,Row Expiration Date,Current Row Indicator
0,6251676,Customer 6251676,6251676,006251676,Rua 6251676,Cidade 6251676,Region 6251676,6251676,+351 006251676,email6251676@gmail.com,2019-04-01,2999-12-31,1
1,6702206,Customer 6702206,6702206,006702206,Rua 6702206,Cidade 6702206,Region 6702206,6702206,+351 006702206,email6702206@gmail.com,2019-04-01,2999-12-31,1
2,6689660,Customer 6689660,6689660,006689660,Rua 6689660,Cidade 6689660,Region 6689660,6689660,+351 006689660,email6689660@gmail.com,2019-04-01,2999-12-31,1
3,8539983,Customer 8539983,8539983,008539983,Rua 8539983,Cidade 8539983,Region 8539983,8539983,+351 008539983,email8539983@gmail.com,2019-04-01,2999-12-31,1
4,8539984,Customer 8539984,8539984,008539984,Rua 8539984,Cidade 8539984,Region 8539984,8539984,+351 008539984,email8539984@gmail.com,2019-04-01,2999-12-31,1


## Dim_Doctor

In [21]:
def make_dim_doctor(df):
    
    df["Doctor Name"] = df["doctor_ref_id"].map(lambda x: "Doctor " + str(x))
    
    dim_doctor = df_train[["doctor_ref_id", "Doctor Name"]]
    dim_doctor["Doctor Key"] = df["doctor_ref_id"]
    dim_doctor["NIF"] = df["doctor_ref_id"].map(lambda x: "00" + str(x))
    dim_doctor["Address"] = df["doctor_ref_id"].map(lambda x: "Rua " + str(x))
    dim_doctor["City"] = df["doctor_ref_id"]. map(lambda x: "Cidade " + str(x))
    dim_doctor["Region"] = df["doctor_ref_id"].map(lambda x: "Region " + str(x))
    dim_doctor["Zip Code"] = df["customer_ref_id"]
    dim_doctor["Phone Number"] = df["customer_ref_id"].map(lambda x: "+351 00" + str(x))
    dim_doctor["Address E-mail"] = df["customer_ref_id"].map(lambda x: "email" + str(x) + "@gmail.com")
    dim_doctor["Row Effective Date"] = df["Date"]
    dim_doctor["Row Expiration Date"] = "2999-12-31"
    dim_doctor["Current Row Indicator"] = 1
    
    
    dim_doctor = dim_doctor.drop_duplicates(subset=["doctor_ref_id"], keep='first')
    
    df.drop(["Doctor Name", "Date"], axis=1, inplace=True)
    
    return dim_doctor

In [22]:
dim_doctor = make_dim_doctor(df_train)

In [23]:
dim_doctor.shape

(72099, 13)

In [24]:
dim_doctor.head()

,doctor_ref_id,Doctor Name,Doctor Key,NIF,Address,City,Region,Zip Code,Phone Number,Address E-mail,Row Effective Date,Row Expiration Date,Current Row Indicator
0,6628790,Doctor 6628790,6628790,006628790,Rua 6628790,Cidade 6628790,Region 6628790,6251676,+351 006251676,email6251676@gmail.com,2019-04-01,2999-12-31,1
1,7298144,Doctor 7298144,7298144,007298144,Rua 7298144,Cidade 7298144,Region 7298144,6702206,+351 006702206,email6702206@gmail.com,2019-04-01,2999-12-31,1
2,6478435,Doctor 6478435,6478435,006478435,Rua 6478435,Cidade 6478435,Region 6478435,6689660,+351 006689660,email6689660@gmail.com,2019-04-01,2999-12-31,1
3,6755214,Doctor 6755214,6755214,006755214,Rua 6755214,Cidade 6755214,Region 6755214,8539983,+351 008539983,email8539983@gmail.com,2019-04-01,2999-12-31,1
5,6489075,Doctor 6489075,6489075,006489075,Rua 6489075,Cidade 6489075,Region 6489075,5897801,+351 005897801,email5897801@gmail.com,2019-04-01,2999-12-31,1


## Dim_PaymentMethod

In [31]:
def make_dim_payment_method(df):
    
    df["payment_ref_id"] = df["payment_method"]\
                                .map({"cash":0, "paymt":1, "card":2, "cheque":3, "phonepe":4, "upi":5})
    
    dim_payment_method = pd.DataFrame({"payment_ref_id":[0,1,2,3,4,5],
                                       "payment_description": ["cash", "paymt", "card", "cheque", "phonepe", "upi"]})
    
    df.drop(["payment_method"], axis=1, inplace=True)
    
    return dim_payment_method

In [32]:
dim_payment_method = make_dim_payment_method(df_train)

In [33]:
dim_payment_method.shape

(6, 2)

In [34]:
dim_payment_method.head()

,payment_ref_id,payment_description
0,0,cash
1,1,paymt
2,2,card
3,3,cheque
4,4,phonepe


## Vendas_fact

In [35]:
df_train.head()

,bill_ref_id,store_ref_id,customer_ref_id,doctor_ref_id,num_drugs_bill,total_quantity_bill,mrp_bill,total_spend_bill,return_value_bill,returned_quantity_bill,quantity_ethical,quantity_generic,quantity_surgical,quantity_ayurvedic,quantity_general,quantity_otc,quantity_chronic,quantity_acute,quantity_h1,DateID,payment_ref_id
0,7562292,468,6251676,6628790,3,12,394.60,146.92,0.0,0,0,11,1,0,0,0,10,2,0,20190401,0.0
1,7562335,468,6702206,7298144,1,1,50.50,26.04,0.0,0,0,1,0,0,0,0,0,1,0,20190401,0.0
2,7562354,468,6689660,6478435,3,7,708.50,330.19,0.0,0,0,7,0,0,0,0,0,7,0,20190401,0.0
3,7562366,468,8539983,6755214,1,1,52.32,26.00,0.0,0,0,1,0,0,0,0,0,1,0,20190401,0.0
4,7562370,468,8539984,7298144,1,1,18.00,14.40,0.0,0,0,1,0,0,0,0,0,1,0,20190401,0.0


## Salvar Todos os Arquivos

In [36]:
df_train.to_csv("bill_fact.csv")

In [37]:
dim_store.to_csv("dim_store.csv")

In [38]:
dim_doctor.to_csv("dim_doctor.csv")

In [39]:
dim_customer.to_csv("dim_customer.csv")

In [40]:
dim_payment_method.to_csv("dim_payment_method.csv")